In [1]:
%matplotlib inline
import galpy
import numpy as np
import scipy as sp
from matplotlib.pylab import plt, xlim, ylim
from astropy import units as u

from galpy.df import streamdf
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014, IsochronePotential
from galpy.actionAngle import actionAngleIsochroneApprox, estimateBIsochrone
from galpy.util import conversion #for unit conversions

In [2]:
#Parameter Block
sigv_Leo = 9.2 #km/s
mw = MWPotential2014 
orb = Orbit.from_name('Leo I')
ts = np.linspace(0,-2,2001)*u.Gyr

In [3]:
#Estimate the scale length of Isochrone Potential
orb.integrate(ts,mw)
b_fits = estimateBIsochrone(mw, orb.R(ts), orb.z(ts))
print(b_fits)

[ 7.144499   18.69086399 30.27207178]


In [ ]:
#Run streamdf for multple values of b
bs = np.linspace(b_fits[0],b_fits[2],260)
b_true = []
for r in bs:
    aAI_mw= actionAngleIsochroneApprox(pot=mw, b=r)
    try:
        sdf= streamdf(sigv_Leo/220., progenitor=orb, pot=mw, aA=aAI_mw,leading=False,
              nTrackChunks=10, nTrackIterations=2, tdisrupt=2/conversion.time_in_Gyr(220.,8.),
                     vo=220, ro=8, R0=8)
    except np.linalg.LinAlgError:
        print(f"{r} didn't work!")
    except ValueError:
        print(f"{r} produced a value error")
    else:
        print(f"No nans/infs reported. {r} works!")
        b_true.append(r)

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:236: RuntimeWarning: invalid value encountered in sqrt
  Jr= self.amp/numpy.sqrt(-2.*E)\

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:239: RuntimeWarning: invalid value encountered in power
  Omegar= (-2.*E)**1.5/self.amp

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:263: RuntimeWarning: invalid value encountered in sqrt
  tan11= numpy.arctan(numpy.sqrt((1.+e)/(1.-e))*numpy.tan(0.5*eta))

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:264: RuntimeWarning: invalid value encountered in sqrt
  tan12= numpy.arctan(numpy.sqrt((1.+e+2.*self.b/c)/(1.-e+2.*self.b/c))*numpy.tan(0.5*eta))

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/numpy/linalg/lin

In [ ]:
# #Plots average location of track
# sdf.plotTrack(d1='r',d2='z',interp=True,color='red',spread=2, lw=2.,scaleToPhysical=True, label="Stream")

# #Turning off interpolation plots points that were explicitly calculated
# sdf.plotTrack(d1='r',d2='z',interp=False,color='yellow',spread=0,overplot=True,ls='none',marker='o',scaleToPhysical=True)

# #We also plot the progenitory to compare
# sdf.plotProgenitor(d1='r',d2='z',color='k', overplot=True, scaleToPhysical=True, label="Leo I")
# plt.legend()

# plt.xlim(10,300)
# plt.ylim(-200,500)